dataset  
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_5.json.gz

https://github.com/ShopRunner/jupyter-notify
```
pip install jupyternotify
```

In [94]:
import tensorflow as tf
import numpy as np
import re
import os
import time
import datetime
from tensorflow.contrib.learn import preprocessing
import pickle
import pandas as pd
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [95]:
class TextCNN:

    def __init__(
      self, sequence_length, num_classes, vocab_size, embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):


        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        l2_loss = tf.constant(0.0)


        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

 
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(self.embedded_chars_expanded,W,strides=[1, 1, 1, 1],padding="VALID", name="conv")
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                pooled = tf.nn.max_pool(h, ksize=[1, sequence_length - filter_size + 1, 1, 1],strides=[1, 1, 1, 1],padding='VALID',name="pool")
                pooled_outputs.append(pooled)


        num_filters_total = num_filters * len(filter_sizes)
        print(num_filters_total)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        

        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
            
                            # add@@@@@@@@
            features_size = 5
            self.features = tf.Variable(tf.truncated_normal([num_filters_total, features_size], stddev=0.1))
            # 384 * 5
            self.f_h = tf.matmul(self.h_drop, self.features, )
              # (None * 383) *(384 * 5) 
            #print(self.h_drop) #None * 384 
            #print(self.f_h) # None * 5



        with tf.name_scope("output"):
            W = tf.get_variable("W", shape=[features_size, num_classes], initializer=tf.contrib.layers.xavier_initializer())
            #W = tf.get_variable("W", shape=[num_filters_total, num_classes], initializer=tf.contrib.layers.xavier_initializer())
            

            
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            #self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.scores = tf.nn.xw_plus_b(self.f_h, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [96]:
# Parameters
# ==================================================

# Data loading params
dev_sample_percentage = 0.07#0.0005

#positive_data_file = "data/amazon/book_pos.txt"#"./data/rt-polarity.pos"
#negative_data_file = "data/amazon/book_neg.txt"#"./data/rt-polarity.neg" #"Data source for the negative data.")
positive_data_file = "data/chABSA/pos.txt"
negative_data_file = "data/chABSA/neg.txt"

# Model Hyperparameters
embedding_dim = 128     #, "Dimensionality of character embedding (default: 128)")
filter_sizes = "3,4,5"        #, "Comma-separated filter sizes (default: '3,4,5')")
num_filters=128               #, "Number of filters per filter size (default: 128)")
dropout_keep_prob=0.5 #, "Dropout keep probability (default: 0.5)")
l2_reg_lambda=0.0          #, "L2 regularization lambda (default: 0.0)")

# Training parameters
batch_size=64                    #, "Batch Size (default: 64)")
num_epochs=200              #, "Number of training epochs (default: 200)")
evaluate_every=100         #, "Evaluate model on dev set after this many steps (default: 100)")
num_checkpoints=5          #, "Number of checkpoints to store (default: 5)")

# Misc Parameters
allow_soft_placement=True    #, "Allow device soft device placement")
log_device_placement=False  #, "Log placement of ops on devices")

#FLAGS = tf.flags.FLAGS
#FLAGS._parse_flags()
#print("\nParameters:")
#for attr, value in sorted(FLAGS.__flags.items()):
#    print("{}={}".format(attr.upper(), value))
#print("")

In [97]:
def load_data_and_labels(positive_data_file, negative_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(open(positive_data_file, "r").readlines())
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = list(open(negative_data_file, "r").readlines())
    negative_examples = [s.strip() for s in negative_examples]
    # Split by words
    x_text = positive_examples + negative_examples
    #x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]

In [98]:
%%notify
x_text, y = load_data_and_labels(positive_data_file, negative_data_file)

<IPython.core.display.Javascript object>

In [99]:
len(x_text)

2830

In [100]:
x_text[0]

'当連結会計年度の世界経済は、米国を中心に景気は概ね堅調に推移しましたが、英国のEU離脱問題による影響のほか、中国をはじめとする新興国経済の減速懸念や米国新政権の政策動向の不確実性など、景気の先行きは不透明な状況が続きました'

In [101]:
#x_text = x_text[:20000]
#y = y[:20000]

In [102]:
#x_text = ["aaaa aa a", "b bbb b bbb d", "あ"]

In [103]:
lang = "Ja" #"En"

In [104]:
%%notify

if lang == "En":
    max_document_length = max([len(x.split(" ")) for x in x_text])
elif lang == "Ja":
    max_document_length = max([len(x) for x in x_text])
print("max document length", max_document_length)
vocab_processor = preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

max document length 401


<IPython.core.display.Javascript object>

In [105]:
x

array([[    1,     2,     3, ...,     0,     0,     0],
       [    6,     7,     8, ...,     0,     0,     0],
       [   10,    11,    12, ...,     0,     0,     0],
       ...,
       [ 7612, 11587, 11588, ...,     0,     0,     0],
       [11580, 11593,    38, ...,     0,     0,     0],
       [ 8365, 11599, 11600, ...,     0,     0,     0]])

In [106]:
%%notify
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

dev_sample_index = -1 * int(dev_sample_percentage * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

del x, y, x_shuffled, y_shuffled

print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Vocabulary Size: 11602
Train/Dev split: 2632/198


<IPython.core.display.Javascript object>

In [108]:
print(x_train.shape)

(2632, 401)


In [109]:
def save_pickle():
    chunk_size =  int(len(x_train)/10)
    print(chunk_size)
    for i in range(0, len(x_train), chunck_size):
        print(i)
        end = i+chunck_size
        if end < len(x_train):
            chunk = x_train[i:  end]
        else:
            chunk = x_train[i:]
            end = len(x_train)
        with open("data/x_train_{}.pkl".format(end), "wb") as f:
            pickle.dump(chunk, f, protocol = 4)

In [110]:
def load_x_train():
    all_size = 999000
    chunk_size = 99900
    x_train =[]
    for i in range(0, all_size, chunck_size):
        print(i)
        end = i+chunk_size
        if end > len(reviews):
            end = all_size
        with open("data/x_train_{}.pkl".format(end), "rb") as f:
            x_train += pickle.load(f)
        return x_train
#x_train = load_x_train()

In [111]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [112]:
sequence_length = x_train.shape[1]
num_classes = y_train.shape[1]
vocab_size = len(vocab_processor.vocabulary_)
embedding_size = embedding_dim
filter_sizes = list(map(int, filter_sizes.split(",")))
num_filters = num_filters
l2_reg_lambda = l2_reg_lambda


In [113]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=allow_soft_placement,
      log_device_placement=log_device_placement)
    sess = tf.Session(config=session_conf)
    
    with sess.as_default():
        
        cnn = TextCNN(sequence_length, num_classes, vocab_size, embedding_size, filter_sizes, num_filters, l2_reg_lambda)


        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)


        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)


        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))


        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)


        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)


        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

    
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

  
        vocab_processor.save(os.path.join(out_dir, "vocab"))


        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)


        batches = batch_iter(list(zip(x_train, y_train)), batch_size, num_epochs)
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % evaluate_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

384
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; us

2018-04-29T21:50:01.903191: step 95, loss 0.595901, acc 0.78125
2018-04-29T21:50:02.392256: step 96, loss 0.970025, acc 0.703125
2018-04-29T21:50:02.877598: step 97, loss 0.676788, acc 0.78125
2018-04-29T21:50:03.380744: step 98, loss 0.560508, acc 0.78125
2018-04-29T21:50:03.867058: step 99, loss 0.609015, acc 0.71875
2018-04-29T21:50:04.368395: step 100, loss 0.522023, acc 0.703125

Evaluation:
2018-04-29T21:50:04.813833: step 100, loss 0.67178, acc 0.656566

Saved model checkpoint to /Users/tdual/Workspace/char_level_cnn/runs/1525006157/checkpoints/model-100

2018-04-29T21:50:05.410248: step 101, loss 1.23359, acc 0.5625
2018-04-29T21:50:05.905996: step 102, loss 0.925395, acc 0.71875
2018-04-29T21:50:06.385362: step 103, loss 0.481425, acc 0.78125
2018-04-29T21:50:06.875433: step 104, loss 0.746831, acc 0.78125
2018-04-29T21:50:07.364388: step 105, loss 0.844505, acc 0.6875
2018-04-29T21:50:07.847782: step 106, loss 0.859016, acc 0.65625
2018-04-29T21:50:08.330158: step 107, loss 0

2018-04-29T21:50:59.120318: step 216, loss 0.377165, acc 0.828125
2018-04-29T21:50:59.666487: step 217, loss 0.257384, acc 0.875
2018-04-29T21:51:00.149172: step 218, loss 0.465991, acc 0.796875
2018-04-29T21:51:00.657572: step 219, loss 0.264236, acc 0.9375
2018-04-29T21:51:01.130342: step 220, loss 0.428461, acc 0.8125
2018-04-29T21:51:01.683820: step 221, loss 0.39818, acc 0.8125
2018-04-29T21:51:02.183035: step 222, loss 0.430285, acc 0.796875
2018-04-29T21:51:02.666151: step 223, loss 0.350816, acc 0.796875
2018-04-29T21:51:03.162241: step 224, loss 0.350252, acc 0.859375
2018-04-29T21:51:03.625020: step 225, loss 0.242559, acc 0.875
2018-04-29T21:51:04.084586: step 226, loss 0.349404, acc 0.875
2018-04-29T21:51:04.553609: step 227, loss 0.274115, acc 0.859375
2018-04-29T21:51:05.086750: step 228, loss 0.168127, acc 0.921875
2018-04-29T21:51:05.539764: step 229, loss 0.401183, acc 0.875
2018-04-29T21:51:06.021054: step 230, loss 0.285158, acc 0.875
2018-04-29T21:51:06.484579: step

2018-04-29T21:51:57.105325: step 340, loss 0.204172, acc 0.890625
2018-04-29T21:51:57.552305: step 341, loss 0.201019, acc 0.890625
2018-04-29T21:51:58.011172: step 342, loss 0.204182, acc 0.921875
2018-04-29T21:51:58.460016: step 343, loss 0.21414, acc 0.890625
2018-04-29T21:51:58.909238: step 344, loss 0.198356, acc 0.9375
2018-04-29T21:51:59.361391: step 345, loss 0.174124, acc 0.96875
2018-04-29T21:51:59.809924: step 346, loss 0.193945, acc 0.921875
2018-04-29T21:52:00.260133: step 347, loss 0.320045, acc 0.859375
2018-04-29T21:52:00.712180: step 348, loss 0.260106, acc 0.875
2018-04-29T21:52:01.157157: step 349, loss 0.117793, acc 0.96875
2018-04-29T21:52:01.617735: step 350, loss 0.166817, acc 0.953125
2018-04-29T21:52:02.072322: step 351, loss 0.282215, acc 0.859375
2018-04-29T21:52:02.536888: step 352, loss 0.1942, acc 0.921875
2018-04-29T21:52:02.991920: step 353, loss 0.135022, acc 0.953125
2018-04-29T21:52:03.436146: step 354, loss 0.212521, acc 0.953125
2018-04-29T21:52:03.

2018-04-29T21:52:51.926108: step 463, loss 0.132297, acc 0.9375
2018-04-29T21:52:52.335542: step 464, loss 0.157008, acc 0.9375
2018-04-29T21:52:52.765495: step 465, loss 0.15858, acc 0.953125
2018-04-29T21:52:53.217267: step 466, loss 0.123916, acc 0.953125
2018-04-29T21:52:53.659942: step 467, loss 0.114946, acc 0.9375
2018-04-29T21:52:54.105702: step 468, loss 0.0976712, acc 0.96875
2018-04-29T21:52:54.559951: step 469, loss 0.181541, acc 0.9375
2018-04-29T21:52:55.016482: step 470, loss 0.300778, acc 0.9375
2018-04-29T21:52:55.456144: step 471, loss 0.0830937, acc 0.984375
2018-04-29T21:52:55.904474: step 472, loss 0.0902132, acc 0.96875
2018-04-29T21:52:56.352255: step 473, loss 0.0800936, acc 1
2018-04-29T21:52:56.791262: step 474, loss 0.0800686, acc 0.96875
2018-04-29T21:52:57.244170: step 475, loss 0.11754, acc 0.953125
2018-04-29T21:52:57.694864: step 476, loss 0.0878993, acc 0.984375
2018-04-29T21:52:58.143363: step 477, loss 0.0613231, acc 0.984375
2018-04-29T21:52:58.59009

...

2018-04-29T22:48:01.548684: step 7834, loss 1.73407e-06, acc 1
2018-04-29T22:48:02.011059: step 7835, loss 3.33949e-06, acc 1
2018-04-29T22:48:02.468867: step 7836, loss 9.76018e-07, acc 1
2018-04-29T22:48:02.902024: step 7837, loss 2.88709e-07, acc 1
2018-04-29T22:48:03.350829: step 7838, loss 2.38061e-05, acc 1
2018-04-29T22:48:03.792670: step 7839, loss 2.3176e-05, acc 1
2018-04-29T22:48:04.239877: step 7840, loss 5.51341e-07, acc 1
2018-04-29T22:48:04.687042: step 7841, loss 5.8107e-06, acc 1
2018-04-29T22:48:05.131541: step 7842, loss 0.0170045, acc 0.984375
2018-04-29T22:48:05.583525: step 7843, loss 2.64118e-06, acc 1
2018-04-29T22:48:06.034648: step 7844, loss 4.01753e-06, acc 1
2018-04-29T22:48:06.484905: step 7845, loss 3.34139e-06, acc 1
2018-04-29T22:48:06.937117: step 7846, loss 3.12348e-06, acc 1
2018-04-29T22:48:07.386236: step 7847, loss 1.79743e-06, acc 1
2018-04-29T22:48:07.836564: step 7848, loss 0.000969393, acc 1
2018-04-29T22:48:08.279687: step 7849, loss 3.88796e

2018-04-29T22:48:58.410477: step 7962, loss 4.14571e-06, acc 1
2018-04-29T22:48:58.852028: step 7963, loss 5.69686e-06, acc 1
2018-04-29T22:48:59.298011: step 7964, loss 1.88127e-07, acc 1
2018-04-29T22:48:59.751126: step 7965, loss 7.09659e-07, acc 1
2018-04-29T22:49:00.203008: step 7966, loss 2.08051e-06, acc 1
2018-04-29T22:49:00.649663: step 7967, loss 4.65382e-05, acc 1
2018-04-29T22:49:01.097856: step 7968, loss 6.29569e-07, acc 1
2018-04-29T22:49:01.552489: step 7969, loss 3.63215e-07, acc 1
2018-04-29T22:49:02.009662: step 7970, loss 2.07123e-06, acc 1
2018-04-29T22:49:02.452065: step 7971, loss 2.6822e-07, acc 1
2018-04-29T22:49:02.884553: step 7972, loss 2.40092e-06, acc 1
2018-04-29T22:49:03.345821: step 7973, loss 1.93709e-06, acc 1
2018-04-29T22:49:03.794085: step 7974, loss 1.12131e-06, acc 1
2018-04-29T22:49:04.230854: step 7975, loss 2.8926e-06, acc 1
2018-04-29T22:49:04.667524: step 7976, loss 1.66516e-06, acc 1
2018-04-29T22:49:05.115604: step 7977, loss 1.32143e-05, 

2018-04-29T22:49:55.479480: step 8090, loss 3.27813e-06, acc 1
2018-04-29T22:49:55.928971: step 8091, loss 2.20157e-06, acc 1
2018-04-29T22:49:56.371050: step 8092, loss 3.55553e-06, acc 1
2018-04-29T22:49:56.817396: step 8093, loss 1.45315e-05, acc 1
2018-04-29T22:49:57.265560: step 8094, loss 1.29824e-06, acc 1
2018-04-29T22:49:57.720359: step 8095, loss 0.000371194, acc 1
2018-04-29T22:49:58.165134: step 8096, loss 2.51593e-05, acc 1
2018-04-29T22:49:58.625003: step 8097, loss 1.61334e-05, acc 1
2018-04-29T22:49:59.077337: step 8098, loss 1.80111e-06, acc 1
2018-04-29T22:49:59.516957: step 8099, loss 3.7585e-06, acc 1
2018-04-29T22:49:59.963632: step 8100, loss 1.63348e-06, acc 1

Evaluation:
2018-04-29T22:50:00.305327: step 8100, loss 3.47359, acc 0.737374

Saved model checkpoint to /Users/tdual/Workspace/char_level_cnn/runs/1525006157/checkpoints/model-8100

2018-04-29T22:50:00.877540: step 8101, loss 8.43774e-07, acc 1
2018-04-29T22:50:01.340949: step 8102, loss 8.074e-05, acc 1


2018-04-29T22:50:51.549469: step 8215, loss 5.70431e-06, acc 1
2018-04-29T22:50:51.997485: step 8216, loss 4.60071e-07, acc 1
2018-04-29T22:50:52.453492: step 8217, loss 1.01774e-05, acc 1
2018-04-29T22:50:52.890978: step 8218, loss 1.71172e-06, acc 1
2018-04-29T22:50:53.338175: step 8219, loss 3.54071e-06, acc 1
2018-04-29T22:50:53.789105: step 8220, loss 2.31659e-05, acc 1
2018-04-29T22:50:54.236470: step 8221, loss 5.42306e-05, acc 1
2018-04-29T22:50:54.689992: step 8222, loss 1.14153e-05, acc 1
2018-04-29T22:50:55.140203: step 8223, loss 5.02911e-07, acc 1
2018-04-29T22:50:55.588076: step 8224, loss 4.38633e-06, acc 1
2018-04-29T22:50:56.036118: step 8225, loss 1.47148e-07, acc 1
2018-04-29T22:50:56.490414: step 8226, loss 6.48194e-07, acc 1
2018-04-29T22:50:56.940122: step 8227, loss 5.1595e-07, acc 1
2018-04-29T22:50:57.387688: step 8228, loss 5.2154e-08, acc 1
2018-04-29T22:50:57.824175: step 8229, loss 9.68574e-08, acc 1
2018-04-29T22:50:58.273270: step 8230, loss 0.00266407, a

2018-04-29T22:51:48.249184: step 8343, loss 1.22144e-05, acc 1
2018-04-29T22:51:48.693485: step 8344, loss 1.07287e-06, acc 1
2018-04-29T22:51:49.128421: step 8345, loss 7.07804e-08, acc 1
2018-04-29T22:51:49.577433: step 8346, loss 1.24608e-06, acc 1
2018-04-29T22:51:50.019432: step 8347, loss 2.03582e-06, acc 1
2018-04-29T22:51:50.461103: step 8348, loss 1.69372e-05, acc 1
2018-04-29T22:51:50.914139: step 8349, loss 2.38573e-05, acc 1
2018-04-29T22:51:51.366046: step 8350, loss 4.08249e-05, acc 1
2018-04-29T22:51:51.812188: step 8351, loss 1.13061e-06, acc 1
2018-04-29T22:51:52.249980: step 8352, loss 1.37835e-07, acc 1
2018-04-29T22:51:52.689554: step 8353, loss 2.48392e-05, acc 1
2018-04-29T22:51:53.150354: step 8354, loss 0.000160816, acc 1
2018-04-29T22:51:53.587457: step 8355, loss 1.11943e-06, acc 1
2018-04-29T22:51:54.036971: step 8356, loss 4.63796e-07, acc 1
2018-04-29T22:51:54.496095: step 8357, loss 6.89177e-08, acc 1
2018-04-29T22:51:54.597427: step 8358, loss 2.47357e-06

Using clean_str  
```
Evaluation:
2018-04-17T10:40:20.162299: step 5000, loss 0.161694, acc 0.942
```

In [ ]:
tf.train.get_checkpoint_state('./runs/1523936751/checkpoints')

In [ ]:
def get_feature(x):
    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=allow_soft_placement,
          log_device_placement=log_device_placement)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=embedding_dim,
                filter_sizes=list(map(int, filter_sizes.split(","))),
                num_filters=num_filters,
                l2_reg_lambda=l2_reg_lambda)

            saver = tf.train.Saver()
            sess.run(tf.global_variables_initializer())

            saver.restore(sess, "./runs/1523936751/checkpoints/model-77900")

            feed_dict = {
                  cnn.input_x: x,
                  cnn.dropout_keep_prob: 1.0
                }
            feature_5, feature_2 = sess.run([cnn.f_h, cnn.scores ], feed_dict=feed_dict)
    return feature_5, feature_2


In [ ]:
review = list(open("data/amazon/rating_5.txt", "r").readlines())
review = [s.strip().split("") for s in review]

In [ ]:
y = []
x = []
for r in review:
    l = r.split(":::::")
    y.append(float(l[0]))
    x.append(l[1])

In [ ]:
vocab_processor = preprocessing.VocabularyProcessor.restore(os.path.join("runs/1523936751", "vocab"))

In [ ]:
x[0]

In [ ]:
y[0]

In [ ]:
x = np.array(list(vocab_processor.fit_transform(x)))
y = np.array(y)

In [ ]:
len(x)

In [ ]:
x[0]

In [ ]:
feature_5 ,feature_2 = get_feature(x[:100])

In [ ]:
feature_5[0]

In [ ]:
feature_5[1]

In [ ]:
s = 0
chunk_size = 100
for i in range(chunk_size, len(x)+chunk_size , chunk_size):
    feature_5 ,feature_2 = get_feature(x[:i])
    for f,r in zip(feature_5, y[:i]):
        s  += int(np.argmax(f) == r)
    print(s)
s

In [ ]:
np.argmax(feature_5)

In [ ]:
feature_2 [0]  #[neg, pos]